## Learning Outcomes:

1. Connecting to our database using SQLAlchemy
1. SQLite
1. The SQLAlchemy ORM
1. Working with the Session object


In [ ]:
# Prerequisites - Install SQLAlchemy
%pip install SQLAlchemy

In [ ]:
# Required imports
from sqlalchemy import create_engine, text
connection_string = "sqlite:///orm1.db"
engine = create_engine(connection_string, echo=True)

# Object-relational mapping (ORM)

Typically, when working with Python, we prefer to work with objects rather than rows in a database table.

This is where the idea of ORM comes in. ORM stands for Object-Relational Mapping. It's a technique whereby we can define Python classes called "models" that are mapped to database tables.

When using an ORM, such as SQLAlchemy, we can work with these Python objects, called "model instances", and have each correspond to a row.

The ORM will then help us translate our Python operations into the appropriate SQL queries.

In [ ]:
# To work with the ORM, we need some additional imports
from sqlalchemy import ForeignKey, Integer, String
from sqlalchemy.orm import Session, declarative_base, mapped_column, relationship

In [ ]:
# We then need to create a base class for our models to inherit from

Base = declarative_base()

In [ ]:
# Let us start with a very simple class - a Car
class Car(Base):
    __tablename__ = "car" # The name of the table in the DB
    id = mapped_column(Integer, primary_key=True)
    model = mapped_column(String(30), nullable=False)
    year = mapped_column(Integer, nullable=False)
    colour = mapped_column(String(30))

    def __str__(self):
        return f"A {self.colour} {self.year} {self.model}"

# Note that it's a regular Python class, but with class-level attributes that describe the columns.
# Each column then is an instance of a SQLAlchemy type, which is a Python class that represents a DB type.

# The basic types are documented in:
# https://docs.sqlalchemy.org/en/20/core/type_basics.html#generic-camelcase-types

In [ ]:
# The table won't be created yet. To create it, we need to run:
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy import create_engine, text
connection_string = "sqlite:///orm2.db"
engine = create_engine(connection_string, echo=True)

from sqlalchemy import Integer, String, ForeignKey
from sqlalchemy.orm import Session, DeclarativeBase, Mapped, mapped_column, relationship

class Base(DeclarativeBase):
    pass

class Owner(Base):
    __tablename__ = "owner"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50), nullable=False)

    #one-to-many relationship
    cars: Mapped[list["Car"]] = relationship(
        back_populates="owner", # link to Car.owner
        cascade="all, delete-orphan"
    )

    def __str__(self) -> str:
        return self.name


class Car(Base):
    __tablename__ = "car"
    id: Mapped[int] = mapped_column(primary_key=True)
    model: Mapped[str] = mapped_column(String(30), nullable=False)
    year: Mapped[int] = mapped_column(nullable=False)
    colour: Mapped[str | None] = mapped_column(String(30))
    owner_id: Mapped[int] = mapped_column(ForeignKey("owner.id"))

    #link back to Owner
    owner: Mapped["Owner"] = relationship(back_populates="cars") # link to owner.cars

    def __str__(self) -> str:
        return f"{self.id}, {self.model}, {self.year}, {self.colour}, {self.owner_id}, {self.owner}"

Base.metadata.create_all(engine)

In [ ]:
# let us create some owners objects
anand = Owner(name="Anand")
dan = Owner(name="Daniel")
print(anand)
print(dan)

# SQLAlchemy Sessions

SQLAlchemy's ORM includes a session management system that handles the interactions between your Python objects and the database. 

A session is a higher-level abstraction than a connection, and it's the recommended way to interact with the database when using the ORM. Sessions allow us to combine multiple operations into a single transaction, and they also provide a way to keep track of changes to objects.

In [ ]:
# store the owner objects in the table
with Session(engine) as session:
    session.add(anand)
    session.add(dan)
    session.commit()

In [ ]:
# check that the owner objects were added to the table
with Session(engine) as session:
    owners = session.query(Owner).all()

for owner in owners:
    print(owner)

In [ ]:
# let us create some car objects
car1 = Car(model="Ford", year=2009, colour="Sea grey", owner=owners[0])
car2 = Car(model="Tesla", year=2020, colour="Red", owner=owners[1])
print(car1)
print(car2)

In [ ]:
# let us add cars to the owners
with Session(engine) as session:
    session.add(car1)
    session.add(car2)
    session.commit()

In [ ]:
engine.echo = False
# let us check if the cars were added to the database
with Session(engine) as session:
    cars = session.query(Car).all()
    for car in cars:
        print(car)



In [ ]:
engine.echo = False
# add more objects
car3 = Car(model="Citreon", year=2023, colour="green", owner=owners[0])
car4 = Car(model="Volkswagon", year=2024, colour="maroon", owner=owners[1])
with Session(engine) as session:    
    session.add(car3)
    session.add(car4)
    session.commit()

In [ ]:
# get all the cars that belong to an owner
with Session(engine) as session:
    anand = session.get(Owner, 1) # Get owner with pk=1    
    # get anand's cars
    print("Anand's cars: ")
    for car in anand.cars:
        print(car)
    
    dan = session.get(Owner, 2)
    # get dan's cars
    print("Dan's cars: ")
    for car in dan.cars:
        print(car)
    

In [ ]:
# now given a car get its owner
with Session(engine) as session:
    car1 = session.get(Car, 1)
    print(car1.owner)

    car2 = session.get(Car, 2)
    print(car2.owner)

In [ ]:
# Optional command only required if you wanted to delete the sqlite db file
engine.dispose()

**For your practice, add yourself and your friend(s) in the breakout room as owners, add a few car objects to them, try to fetch the data and play around with things like getting the cars for a given owner, getting the owner given a car, delete a owner and see if the cars get deleted as well
Also, if you have time, practice with other one to many relationships and perhaps other kinds of relationships such as a one to one relationship**
